In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 2.4 MB/s eta 0:00:00a 0:00:01


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


spark = SparkSession.builder\
        .appName("Test_pivot")\
        .getOrCreate()

In [4]:
df = spark.read.csv('csv/ratings.csv', header=True)

In [5]:
df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [6]:
list_users = ['1','2','3','4','5','6','7','8','9']

In [7]:
df_new = df.select(df.userId,df.movieId)
df_new.show()

+------+-------+
|userId|movieId|
+------+-------+
|     1|      1|
|     1|      3|
|     1|      6|
|     1|     47|
|     1|     50|
|     1|     70|
|     1|    101|
|     1|    110|
|     1|    151|
|     1|    157|
|     1|    163|
|     1|    216|
|     1|    223|
|     1|    231|
|     1|    235|
|     1|    260|
|     1|    296|
|     1|    316|
|     1|    333|
|     1|    349|
+------+-------+
only showing top 20 rows



In [8]:
users = df_new.\
  select('userId','movieId').\
  filter(df_new.userId.isin(list_users))

In [9]:
users2 = users.groupBy('userId').agg(count('movieId').alias('movies')).orderBy(users.userId.asc())

In [10]:
users2.show()

+------+------+
|userId|movies|
+------+------+
|     1|   232|
|     2|    29|
|     3|    39|
|     4|   216|
|     5|    44|
|     6|   314|
|     7|   152|
|     8|    47|
|     9|    46|
+------+------+



In [11]:
users_pivot = users.groupBy('userId').pivot('userId').agg(count('movieId').alias('movies')).withColumn('Usuarios',lit('Number of movies')).orderBy(users.userId.asc())
users_pivot.printSchema()

root
 |-- userId: string (nullable = true)
 |-- 1: long (nullable = true)
 |-- 2: long (nullable = true)
 |-- 3: long (nullable = true)
 |-- 4: long (nullable = true)
 |-- 5: long (nullable = true)
 |-- 6: long (nullable = true)
 |-- 7: long (nullable = true)
 |-- 8: long (nullable = true)
 |-- 9: long (nullable = true)
 |-- Usuarios: string (nullable = false)



In [12]:
otro = users_pivot.select(
    sum('1').alias('user 1'),
    sum('2').alias('user 2'),
    sum('3').alias('user 3'),
    sum('4').alias('user 4'),
    sum('5').alias('user 5'),
    sum('6').alias('user 6'),
    sum('7').alias('user 7'),
    sum('8').alias('user 8'),
    sum('9').alias('user 9'))

In [13]:
otro.show()

+------+------+------+------+------+------+------+------+------+
|user 1|user 2|user 3|user 4|user 5|user 6|user 7|user 8|user 9|
+------+------+------+------+------+------+------+------+------+
|   232|    29|    39|   216|    44|   314|   152|    47|    46|
+------+------+------+------+------+------+------+------+------+

